In [ ]:
# 패키지 설치
!pip install yfinance --quiet

In [ ]:
# 패키지 임포트
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
sns.set()

## 데이터셋 준비

총 16개 ETF에 대한 가격 데이터를 다운받아오도록 하겠습니다. 여기서 사용하게 될 데이터는 전부 월간 단위 데이터입니다. 

In [ ]:
tickers = ['XLB', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP', 'XLU', 'XLV', 'XLY']
etf = yf.Tickers(tickers)
data = etf.history(start='2010-01-01', actions=False)
data.drop(['Open', 'High', 'Low', 'Volume'], inplace=True, axis=1)
data = data.droplevel(0, axis=1).resample('W').last()

In [ ]:
# 수익률
rets = data.pct_change().fillna(0)

In [ ]:
# 색깔 팔레트
pal = sns.color_palette('Spectral', len(tickers))

### 동일 비중 포트폴리오

In [ ]:
# 동일 비중 포트폴리오 수익률
port_rets = rets.mean(axis=1)

# 백테스팅 결과
(1 + port_rets).cumprod().plot()

### 변동성 타겟팅 모델

In [ ]:
# 1년 롤링 변동성 계산
rolling_vol = port_rets.rolling(52).std() * np.sqrt(52)
rolling_vol.plot()

In [ ]:
# 변동성 타겟팅을 사용해 투자 가중치 산출
# 타겟 변동성 = 10%
target_vol = 0.1
vt_weights = (target_vol / rolling_vol).fillna(0)

# 레버리지 사용 불가
vt_weights[vt_weights > 1] = 1

# 볼타겟 백테스팅 수익률
tvol_rets = (port_rets * vt_weights.shift()).fillna(0)
tvol_cum_rets = (1 + tvol_rets).cumprod()

# 백테스팅 결과 시각화
plt.figure(figsize=(10, 5))
plt.plot((1 + port_rets).cumprod(), label='EW')
plt.plot(tvol_cum_rets, label=f'Target Vol = {target_vol}')
plt.title('VT Backtest')
plt.xlabel('Date')
plt.ylabel('Return')
plt.legend()

In [ ]:
# 변동성 타겟팅 포트폴리오의 시계열 변동성 시각화
plt.figure(figsize=(10, 5))
plt.plot((tvol_rets.rolling(52).std() * np.sqrt(52)).dropna(), label='1y rolling vol')
plt.axhline((tvol_rets.std() * np.sqrt(52)).mean(), color='r', label=f'average vol={np.round((tvol_rets.std() * np.sqrt(52)).mean(), 3)}')
plt.title('VT Volatility')
plt.xlabel('Date')
plt.ylabel('Volatility')
plt.legend()
plt.show()

In [ ]:
# 현금 보유 비중
1 - vt_weights

In [ ]:
# 투자 비중 데이터프레임 생성
vt_weights_df = pd.concat([vt_weights, 1-vt_weights], axis=1, join='inner')
vt_weights_df.columns = ['portfolio', 'cash']
vt_weights_df

In [ ]:
# 시계열 플랏 시각화
plt.figure(figsize=(20, 5))
plt.stackplot(vt_weights_df.index, vt_weights_df.T, labels=vt_weights_df.columns, colors=['steelblue', 'silver'])
plt.legend(loc='upper left')
plt.title('VT Weights')
plt.xlabel('Date')
plt.ylabel('Weights')

Copyright 2022. 퀀트대디. All rights reserved.